<a href="https://colab.research.google.com/github/aman-pant/stocktracker/blob/main/tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=6aa66b311ea7dd345703ef37fe55305fa1148eb96e845b8e315e5d01dd13cf51
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [2]:
# tracker.py
import pandas as pd
import yfinance as yf
import ta
import os
from datetime import datetime

WATCHLIST = ['BPCL.NS','KARURVYSYA.NS','DMART.NS']

# WATCHLIST = ['BPCL.NS','KARURVYSYA.NS','DMART.NS','AUBANK.NS','DABUR.NS','HDFCBANK.NS','RELIANCE.NS','ASHOKLEY.NS','IRCTC.NS','IDEA.NS','INFY.NS',
#         'BHARTIARTL.NS','HCLTECH.NS','POWERGRID.NS','SBILIFE.NS','SBICARD.NS','ICICIGI.NS','BEL.NS','NTPC.NS','PFC.NS','NBCC.NS',
#         'VBL.NS','TITAN.NS','ICICIBANK.NS','RAIN.NS','TATAMOTORS.NS','ASIANPAINT.NS','INDIACEM.NS','APOLLOTYRE.NS',
#         'RECLTD.NS','FSL.NS','CESC.NS','COCHINSHIP.NS','COALINDIA.NS','EMAMILTD.NS','UFO.NS','JSWENERGY.NS','GRANULES.NS','FRETAIL.NS','IOC.NS',
#         'DEEPAKNTR.NS','MCLEODRUSS.NS','GICHSGFIN.NS','MARICO.NS','SUNPHARMA.NS','SBIN.NS','BAJFINANCE.NS',
#         'DELTACORP.NS','AFFLE.NS','SPENCERS.NS','TATAPOWER.NS','ITC.NS','LUPIN.NS','BANDHANBNK.NS','NCC.NS','BPCL.NS','NETWORK18.NS','ZENSARTECH.NS',
#         'GODREJCP.NS','IGL.NS','RBLBANK.NS','RITES.NS','TRIDENT.NS','YESBANK.NS']
SIP_LOG_PATH = 'data/sip_log.csv'

RSI_BUY_THRESHOLD = 35
RSI_SELL_THRESHOLD = 78

def fetch_stock_data(ticker):
    df = yf.download(ticker, period="2y", interval="1d")
    df['EMA_10'] = df['Close'].ewm(span=10, adjust=False).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['MA_200'] = df['Close'].rolling(window=200).mean()

    # Convert 'Close' column to a 1D Series if it's 2D
    close_series = df['Close']
    if isinstance(close_series, pd.DataFrame) or len(close_series.shape) > 1:
        close_series = close_series.squeeze()  # Converts (n,1) -> (n,)

    # Now use RSIIndicator with the correct 1D Series
    rsi_indicator = ta.momentum.RSIIndicator(close=close_series, window=14)
    df['RSI'] = rsi_indicator.rsi()

    return df.dropna()


def evaluate_signals(ticker, df):
    latest = df.iloc[-1]
    rsi = latest['RSI'].iloc[0]
    close = latest['Close'].iloc[0]
    sma = latest['MA_200'].iloc[0]


    if rsi < RSI_BUY_THRESHOLD:
        action = 'Buy'
    elif rsi > RSI_SELL_THRESHOLD:
        action = 'Sell'
    else:
        action = 'Hold'

    return {
        'Ticker': ticker,
        'Date': latest.name.strftime('%Y-%m-%d'),
        'Close': round(close, 2),
        'RSI': round(rsi, 2),
        'SMA200': round(sma, 2),
        'Action': action
    }

def load_sip_log():
    if os.path.exists(SIP_LOG_PATH):
        return pd.read_csv(SIP_LOG_PATH, parse_dates=['Date'])
    return pd.DataFrame(columns=['Date', 'Ticker', 'Amount'])

def main():
    sip_log = load_sip_log()
    results = []
    for ticker in WATCHLIST:
        df = fetch_stock_data(ticker)
        signal = evaluate_signals(ticker, df)
        results.append(signal)

    print("\n=== Portfolio Actions ===")
    for r in results:
        print(f"{r['Date']} - {r['Ticker']} | Close: {r['Close']} |SMA 200: {r['SMA200']} | RSI: {r['RSI']} -> {r['Action']}")

if __name__ == '__main__':
    main()


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


=== Portfolio Actions ===
2025-05-30 - BPCL.NS | Close: 318.4 |SMA 200: 298.08 | RSI: 57.06 -> Hold
2025-05-30 - KARURVYSYA.NS | Close: 221.69 |SMA 200: 218.81 | RSI: 51.68 -> Hold
2025-05-30 - DMART.NS | Close: 4002.1 |SMA 200: 4112.15 | RSI: 42.41 -> Hold


In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime

WATCHLIST = ['INFY.NS', 'TCS.NS', 'RELIANCE.NS']

def fetch_ema_data(ticker):
    df = yf.download(ticker, period="2y", interval="1d")
    df['EMA_10'] = df['Close'].ewm(span=10, adjust=False).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['MA_200'] = df['Close'].rolling(window=200).mean()
    # Calculate 30-day average volume
    df['Avg_Volume_30'] = df['Volume'].rolling(window=30).mean()

    # Today's volume (most recent day)
    df['Today_Volume'] = df['Volume']

    # Volume indicator: True if today's volume > 110% of 30-day average
    df['High_Volume_Alert'] = df['Today_Volume'] > (df['Avg_Volume_30'] * 1.1)

    # Additional volume metrics for better analysis
    # df['Volume_Ratio'] = df['Volume'] / df['Avg_Volume_30']
    # df['Volume_vs_Avg_Pct'] = ((df['Volume'] - df['Avg_Volume_30']) / df['Avg_Volume_30'] * 100).round(2)
    return df.dropna()

def get_valuation_ratios(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    pe_ratio = info.get('trailingPE', 'N/A')
    pb_ratio = info.get('priceToBook', 'N/A')
    return pe_ratio, pb_ratio

def check_ema_strategy(ticker, df):
    latest = df.iloc[-1]
    close = latest['Close'].iloc[0]
    ema_10 = latest['EMA_10'].iloc[0]
    ema_50 = latest['EMA_50'].iloc[0]
    sma = latest['MA_200'].iloc[0]
    # P/E: ticker.pe_ratio
    # P/B: ticker.pb_ratio
    price_diff = abs(ema_50 - ema_10)
    diff_pct = price_diff / close


    signal = None
    if (ema_50 > ema_10 and diff_pct < 0.05) or (ema_10 > ema_50 and diff_pct < 0.05):
        signal = 'Buy'

    return {
        'Ticker': ticker,
        'Date': latest.name.strftime('%Y-%m-%d'),
        'Close': round(close, 2),
        'EMA_10': round(ema_10, 2),
        'EMA_50': round(ema_50, 2),
        'SMA_200': round(sma, 2),
        'Signal': signal or 'Hold'
    }

def main():
    results = []
    for ticker in WATCHLIST:
        df = fetch_ema_data(ticker)
        result = check_ema_strategy(ticker, df)
        results.append(result)

    df_results = pd.DataFrame(results)
    print("\nEMA Strategy Output:")
    print(df_results)

if __name__ == '__main__':
    main()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


EMA Strategy Output:
        Ticker        Date   Close   EMA_10   EMA_50  SMA_200 Signal
0      INFY.NS  2025-05-30  1562.7  1550.77  1551.76  1755.88    Buy
1       TCS.NS  2025-05-30  3463.4  3499.28  3524.98  3948.20    Buy
2  RELIANCE.NS  2025-05-30  1420.9  1421.37  1356.05  1327.70    Buy


Key Additions:
1. Fundamental Metrics:

P/E Ratio < 15: Reasonable valuation
P/B Ratio < 2: Not overvalued relative to book value
Dividend Yield > 2.5%: Decent income generation
ROE > 15%: Strong profitability
Distance from 52-week low < 30%: Potentially undervalued
RSI < 40: Technically oversold

2. Enhanced Signal Logic:

Value Score (0-6): Counts how many positive signals each stock has
Strong Buy: 3+ positive fundamental/technical signals
Buy: 2+ positive signals or original EMA/volume conditions
Hold: Less than 2 positive signals

3. New Output Columns:

P/E, P/B ratios
Dividend yield and ROE percentages
Distance from 52-week low
RSI indicator
Value score and reasons for signal

4. Improved Analysis:

Separates "Strong Buy" (value + technical confluence) from regular "Buy"
Shows specific reasons for each signal
Better summary with fundamental metrics displayed

This approach identifies genuinely undervalued stocks with strong fundamentals rather than just technical patterns. The Strong Buy signals indicate stocks that are both technically and fundamentally attractive - the highest probability opportunities.  

In [18]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

WATCHLIST = ['INFY.NS', 'TCS.NS', 'RELIANCE.NS','ADANIPOWER.NS','HINDCOPPER.NS','HINDZINC.NS','sdbl.NS','MRPL.NS','REDINGTON.NS','NBCC.NS']

def fetch_ema_data(ticker):
    """Fetch stock data and calculate technical indicators"""
    try:
        df = yf.download(ticker, period="2y", interval="1d", progress=False)
        if df.empty:
            print(f"No data found for {ticker}")
            return None

        df['EMA_10'] = df['Close'].ewm(span=10, adjust=False).mean()
        df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
        df['MA_200'] = df['Close'].rolling(window=200).mean()

        # Calculate 30-day average volume
        df['Avg_Volume_30'] = df['Volume'].rolling(window=50).mean()
        df['Today_Volume'] = df['Volume']
        df['High_Volume_Alert'] = df['Today_Volume'] > (df['Avg_Volume_30'] * 1.4)

        return df.dropna()
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

def calculate_rsi(prices, window=14):
    """Calculate RSI (Relative Strength Index)"""
    try:
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

        # Avoid division by zero
        rs = gain / loss.replace(0, np.inf)
        rsi = 100 - (100 / (1 + rs))
        return rsi
    except Exception as e:
        print(f"Error calculating RSI: {e}")
        return pd.Series([50] * len(prices), index=prices.index)

def safe_extract_value(value):
    """Safely extract a scalar value from pandas Series/DataFrame"""
    if pd.isna(value):
        return None
    if hasattr(value, 'iloc'):
        return value.iloc[0] if len(value) > 0 else None
    if hasattr(value, 'item'):
        return value.item()
    return value

def get_fundamental_data(ticker):
    """Get key fundamental metrics for valuation analysis"""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        # Handle missing data gracefully with safe extraction
        pe_ratio = safe_extract_value(info.get('trailingPE'))
        pb_ratio = safe_extract_value(info.get('priceToBook'))
        dividend_yield = safe_extract_value(info.get('dividendYield', 0))
        if dividend_yield and dividend_yield > 0:
            dividend_yield = dividend_yield * 100
        roe = safe_extract_value(info.get('returnOnEquity'))
        if roe and roe > 0:
            roe = roe * 100

        # Get price data
        current_price = safe_extract_value(info.get('currentPrice'))
        week_52_high = safe_extract_value(info.get('fiftyTwoWeekHigh'))
        week_52_low = safe_extract_value(info.get('fiftyTwoWeekLow'))

        # If current price not available, try to get from history
        if not current_price:
            hist = stock.history(period="5d")
            if not hist.empty:
                current_price = safe_extract_value(hist['Close'].iloc[-1])

        # Calculate distance from 52-week low
        distance_from_low = None
        if current_price and week_52_low and week_52_low > 0:
            distance_from_low = ((current_price - week_52_low) / week_52_low) * 100

        return {
            'pe_ratio': pe_ratio,
            'pb_ratio': pb_ratio,
            'dividend_yield': dividend_yield,
            'roe': roe,
            'distance_from_52w_low': distance_from_low,
            '52w_high': week_52_high,
            '52w_low': week_52_low,
            'current_price': current_price
        }
    except Exception as e:
        print(f"Error fetching fundamentals for {ticker}: {e}")
        return {
            'pe_ratio': None, 'pb_ratio': None, 'dividend_yield': None,
            'roe': None, 'distance_from_52w_low': None,
            '52w_high': None, '52w_low': None, 'current_price': None
        }

def check_ema_strategy(ticker, df):
    """Analyze stock using EMA strategy and fundamental analysis"""
    try:
        if df is None or df.empty:
            return None

        # Get the last row and extract values safely
        latest_row = df.iloc[-1]

        # Use .values to get numpy array, then extract first element
        close = latest_row['Close'].iloc[0]
        ema_10 = latest_row['EMA_10'].iloc[0]
        ema_50 = latest_row['EMA_50'].iloc[0]
        sma_200 = latest_row['MA_200'].iloc[0]
        today_volume = latest_row['Today_Volume'].iloc[0]
        avg_volume_30 = latest_row['Avg_Volume_30'].iloc[0]
        high_volume_alert = latest_row['High_Volume_Alert'].iloc[0]

        # Get the date
        date_str = latest_row.name.strftime('%Y-%m-%d')

        # Get fundamental data
        fundamentals = get_fundamental_data(ticker)

        # Calculate price difference percentage
        price_diff = abs(ema_50 - ema_10)
        diff_pct = price_diff / close if close > 0 else 0

        # Calculate RSI
        rsi_series = calculate_rsi(df['Close'], 14)
        rsi = float(rsi_series.iloc[-1]) if not rsi_series.empty else 50.0

        # Signal generation
        signal = 'Hold'
        signal_reasons = []
        value_score = 0

        # Technical signals
        if diff_pct < 0.05:  # EMAs are converging
            signal = 'Buy'
            signal_reasons.append('EMA Convergence')

        # Volume signal
        if high_volume_alert:
            if signal != 'Buy':
                signal = 'Buy'
            signal_reasons.append('High Volume')

        # Value analysis - use explicit None checks
        pe_ratio = fundamentals['pe_ratio']
        if pe_ratio is not None and pe_ratio < 15:
            value_score += 1
            signal_reasons.append('Low P/E')

        pb_ratio = fundamentals['pb_ratio']
        if pb_ratio is not None and pb_ratio < 4:
            value_score += 1
            signal_reasons.append('Low P/B')

        div_yield = fundamentals['dividend_yield']
        if div_yield is not None and div_yield > 2.5:
            value_score += 1
            signal_reasons.append('Good Dividend')

        roe = fundamentals['roe']
        if roe is not None and roe > 15:
            value_score += 1
            signal_reasons.append('High ROE')

        dist_52w = fundamentals['distance_from_52w_low']
        if dist_52w is not None and dist_52w < 30:
            value_score += 1
            signal_reasons.append('Near 52W Low')

        if rsi < 40:
            value_score += 1
            signal_reasons.append('Oversold RSI')

        # Determine final signal
        if value_score >= 3:
            signal = 'Strong Buy'
        elif value_score >= 2:
            signal = 'Buy'

        # Format output values safely
        def safe_format(value, decimal_places=1):
            if value is None:
                return 'N/A'
            try:
                return round(float(value), decimal_places)
            except:
                return 'N/A'

        def safe_format_percent(value, decimal_places=1):
            if value is None:
                return 'N/A'
            try:
                return f"{float(value):.{decimal_places}f}%"
            except:
                return 'N/A'

        return {
            'Ticker': ticker,
            'Date': date_str,
            'Close': safe_format(close, 2),
            'EMA_10': safe_format(ema_10, 2),
            'EMA_50': safe_format(ema_50, 2),
            'SMA_200': safe_format(sma_200, 2),
            'RSI': safe_format(rsi, 1),
            'P/E': safe_format(pe_ratio, 1),
            'P/B': safe_format(pb_ratio, 1),
            'Div_Yield': safe_format_percent(div_yield, 1),
            'ROE': safe_format_percent(roe, 1),
            'Distance_52W_Low': safe_format_percent(dist_52w, 0),
            'Today_Volume': f"{int(today_volume):,}",
            'Avg_Volume_30': f"{int(avg_volume_30):,}",
            'High_Volume_Alert': '🔴 YES' if high_volume_alert else '🟢 NO',
            'Value_Score': f"{value_score}/6",
            'Signal': signal,
            'Reasons': ', '.join(signal_reasons) if signal_reasons else 'No signals'
        }

    except Exception as e:
        print(f"Error analyzing {ticker}: {e}")
        import traceback
        traceback.print_exc()
        return None

def main():
    """Main function to run the stock analysis"""
    print("Starting stock analysis...")
    print(f"Analyzing stocks: {', '.join(WATCHLIST)}")
    print("-" * 50)

    results = []
    for ticker in WATCHLIST:
        print(f"Processing {ticker}...")
        try:
            df = fetch_ema_data(ticker)
            if df is not None and not df.empty:
                result = check_ema_strategy(ticker, df)
                if result:
                    results.append(result)
                    print(f"✓ {ticker} processed successfully")
                else:
                    print(f"✗ {ticker} analysis failed")
            else:
                print(f"✗ {ticker} data fetch failed")
        except Exception as e:
            print(f"✗ Error processing {ticker}: {str(e)}")
            continue

    if not results:
        print("No results to display. Please check your internet connection and try again.")
        return

    # Create results DataFrame
    df_results = pd.DataFrame(results)

    print("\n" + "="*120)
    print("EMA STRATEGY OUTPUT WITH VOLUME ANALYSIS")
    print("="*120)

    # Display results
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 30)

    print(df_results.to_string(index=False))

    # Summary statistics
    buy_signals = df_results[df_results['Signal'].isin(['Buy', 'Strong Buy'])]
    strong_buy_signals = df_results[df_results['Signal'] == 'Strong Buy']
    high_volume_stocks = df_results[df_results['High_Volume_Alert'].str.contains('YES')]

    print(f"\n📊 SUMMARY:")
    print(f"Total stocks analyzed: {len(df_results)}")
    print(f"Buy signals (all): {len(buy_signals)}")
    print(f"Strong Buy signals: {len(strong_buy_signals)}")
    print(f"High volume alerts: {len(high_volume_stocks)}")

    if len(strong_buy_signals) > 0:
        print(f"\n🎯 STRONG BUY SIGNALS (Value + Technical):")
        for _, stock in strong_buy_signals.iterrows():
            print(f"  • {stock['Ticker']}: ₹{stock['Close']} | Value Score: {stock['Value_Score']} | Reasons: {stock['Reasons']}")

    if len(buy_signals) > 0:
        print(f"\n📈 ALL BUY SIGNALS:")
        for _, stock in buy_signals.iterrows():
            print(f"  • {stock['Ticker']}: ₹{stock['Close']} | P/E: {stock['P/E']} | P/B: {stock['P/B']} | ROE: {stock['ROE']}")

    print("\nAnalysis complete!")

if __name__ == '__main__':
    main()

Starting stock analysis...
Analyzing stocks: INFY.NS, TCS.NS, RELIANCE.NS, ADANIPOWER.NS, HINDCOPPER.NS, HINDZINC.NS, sdbl.NS, MRPL.NS, REDINGTON.NS, NBCC.NS
--------------------------------------------------
Processing INFY.NS...
✓ INFY.NS processed successfully
Processing TCS.NS...
✓ TCS.NS processed successfully
Processing RELIANCE.NS...
✓ RELIANCE.NS processed successfully
Processing ADANIPOWER.NS...
✓ ADANIPOWER.NS processed successfully
Processing HINDCOPPER.NS...
✓ HINDCOPPER.NS processed successfully
Processing HINDZINC.NS...
✓ HINDZINC.NS processed successfully
Processing sdbl.NS...
✓ sdbl.NS processed successfully
Processing MRPL.NS...
✓ MRPL.NS processed successfully
Processing REDINGTON.NS...
✓ REDINGTON.NS processed successfully
Processing NBCC.NS...
✓ NBCC.NS processed successfully

EMA STRATEGY OUTPUT WITH VOLUME ANALYSIS
       Ticker       Date   Close  EMA_10  EMA_50  SMA_200  RSI  P/E   P/B Div_Yield   ROE Distance_52W_Low Today_Volume Avg_Volume_30 High_Volume_Alert

RSI > 75 (indicates overbought)


MACD divergence + high volume


Price > 30–40% above 200-day moving average


🟡 Valuation Triggers:
P/E ratio > 2x industry average


P/B ratio > 2.5 or 3, without a strong change in earnings


Dividend yield drops significantly due to price surge (means price has inflated)




In [14]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

WATCHLIST = ['INFY.NS', 'TCS.NS', 'RELIANCE.NS']

# Industry average P/E ratios (you can adjust these based on current market conditions)
INDUSTRY_PE_AVERAGES = {
    'INFY.NS': 25,    # IT sector average
    'TCS.NS': 25,     # IT sector average
    'RELIANCE.NS': 20  # Oil & Gas/Conglomerate average
}

def fetch_technical_data(ticker):
    """Fetch stock data and calculate technical indicators"""
    try:
        df = yf.download(ticker, period="2y", interval="1d", progress=False)
        if df.empty:
            print(f"No data found for {ticker}")
            return None

        # Handle MultiIndex columns if present
        if isinstance(df.columns, pd.MultiIndex):
            # Flatten the MultiIndex columns
            df.columns = df.columns.droplevel(1)

        # Ensure we have the required columns
        required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            print(f"Missing required columns for {ticker}: {missing_columns}")
            return None

        # Calculate moving averages
        df['MA_200'] = df['Close'].rolling(window=200).mean()
        df['MA_50'] = df['Close'].rolling(window=50).mean()
        df['MA_20'] = df['Close'].rolling(window=20).mean()

        # Calculate MACD
        df['EMA_12'] = df['Close'].ewm(span=12, adjust=False).mean()
        df['EMA_26'] = df['Close'].ewm(span=26, adjust=False).mean()
        df['MACD'] = df['EMA_12'] - df['EMA_26']
        df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
        df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']

        # Calculate volume indicators with error handling
        try:
            df['Avg_Volume_30'] = df['Volume'].rolling(window=30).mean()
            df['Volume_Ratio'] = df['Volume'] / df['Avg_Volume_30']
            df['High_Volume_Alert'] = df['Volume'] > (df['Avg_Volume_30'] * 1.2)  # 20% above average
        except Exception as vol_error:
            print(f"Error calculating volume indicators for {ticker}: {vol_error}")
            # Set default values if volume calculation fails
            df['Avg_Volume_30'] = 0
            df['Volume_Ratio'] = 1
            df['High_Volume_Alert'] = False

        return df.dropna()

    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

def calculate_rsi(prices, window=14):
    """Calculate RSI (Relative Strength Index)"""
    try:
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

        rs = gain / loss.replace(0, np.inf)
        rsi = 100 - (100 / (1 + rs))
        return rsi
    except Exception as e:
        print(f"Error calculating RSI: {e}")
        return pd.Series([50] * len(prices), index=prices.index)

def safe_extract_value(value):
    """Safely extract a scalar value from pandas Series/DataFrame"""
    if pd.isna(value):
        return None
    if hasattr(value, 'iloc'):
        return value.iloc[0] if len(value) > 0 else None
    if hasattr(value, 'item'):
        return value.item()
    return value

def get_fundamental_data(ticker):
    """Get key fundamental metrics for valuation analysis"""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        # Current fundamental data
        pe_ratio = safe_extract_value(info.get('trailingPE'))
        pb_ratio = safe_extract_value(info.get('priceToBook'))
        dividend_yield = safe_extract_value(info.get('dividendYield', 0))
        if dividend_yield and dividend_yield > 0:
            dividend_yield = dividend_yield * 100

        # Get historical dividend yield (1 year ago) for comparison
        hist_1y = stock.history(period="1y")
        if not hist_1y.empty and len(hist_1y) > 250:
            price_1y_ago = safe_extract_value(hist_1y['Close'].iloc[0])
        else:
            price_1y_ago = None

        # Current price data
        current_price = safe_extract_value(info.get('currentPrice'))
        if not current_price:
            hist = stock.history(period="5d")
            if not hist.empty:
                current_price = safe_extract_value(hist['Close'].iloc[-1])

        # Calculate historical dividend yield if we have the data
        hist_div_yield = None
        if price_1y_ago and dividend_yield and current_price:
            # Approximate historical dividend yield
            hist_div_yield = (dividend_yield * current_price / price_1y_ago) if price_1y_ago > 0 else None

        # Get earnings data for growth analysis
        earnings_growth = safe_extract_value(info.get('earningsGrowth'))
        if earnings_growth:
            earnings_growth = earnings_growth * 100

        return {
            'pe_ratio': pe_ratio,
            'pb_ratio': pb_ratio,
            'dividend_yield': dividend_yield,
            'hist_dividend_yield': hist_div_yield,
            'current_price': current_price,
            'earnings_growth': earnings_growth,
            'price_1y_ago': price_1y_ago
        }
    except Exception as e:
        print(f"Error fetching fundamentals for {ticker}: {e}")
        return {
            'pe_ratio': None, 'pb_ratio': None, 'dividend_yield': None,
            'hist_dividend_yield': None, 'current_price': None,
            'earnings_growth': None, 'price_1y_ago': None
        }

def check_macd_divergence(df, lookback=20):
    """Check for MACD bearish divergence"""
    try:
        if len(df) < lookback:
            return False

        recent_data = df.tail(lookback)

        # Check if price is making higher highs but MACD is making lower highs
        price_trend = recent_data['Close'].iloc[-1] > recent_data['Close'].iloc[0]
        macd_trend = recent_data['MACD'].iloc[-1] < recent_data['MACD'].iloc[0]

        # Bearish divergence: price up, MACD down
        return price_trend and macd_trend
    except:
        return False

def check_sell_signals(ticker, df):
    """Analyze stock for sell signals"""
    try:
        if df is None or df.empty:
            return None

        # Get the last row and extract values safely
        latest_row = df.iloc[-1]

       # Extract scalar values directly (no .iloc[0] needed)
        close = latest_row['Close']
        ma_200 = latest_row['MA_200']
        ma_50 = latest_row['MA_50']
        volume = latest_row['Volume']
        avg_volume_30 = latest_row['Avg_Volume_30']
        high_volume_alert = latest_row['High_Volume_Alert']
        volume_ratio = latest_row['Volume_Ratio']

        # MACD data
        macd = latest_row['MACD']
        macd_signal = latest_row['MACD_Signal']
        macd_histogram = latest_row['MACD_Histogram']

        # Get the date - handle both datetime index and regular index
        try:
            if hasattr(latest_row.name, 'strftime'):
                date_str = latest_row.name.strftime('%Y-%m-%d')
            else:
                date_str = str(latest_row.name)
        except:
            date_str = 'N/A'

        # Calculate RSI
        rsi_series = calculate_rsi(df['Close'], 14)
        rsi = float(rsi_series.iloc[-1]) if not rsi_series.empty else 50.0

        # Get fundamental data
        fundamentals = get_fundamental_data(ticker)

        # Check MACD divergence
        macd_divergence = check_macd_divergence(df, 20)

        # Calculate price distance from 200-day MA
        price_above_ma200_pct = ((close - ma_200) / ma_200) * 100 if ma_200 > 0 else 0

        # Signal generation
        signal = 'Hold'
        signal_reasons = []
        risk_score = 0

        # 🔴 TECHNICAL SELL SIGNALS

        # RSI Overbought (RSI > 75)
        if rsi > 75:
            risk_score += 2
            signal_reasons.append('Overbought RSI')
            signal = 'Sell'

        # MACD Divergence + High Volume
        if macd_divergence and high_volume_alert:
            risk_score += 2
            signal_reasons.append('MACD Divergence + High Volume')
            signal = 'Sell'
        elif macd_divergence:
            risk_score += 1
            signal_reasons.append('MACD Divergence')

        # Price significantly above 200-day MA (30-40%+)
        if price_above_ma200_pct > 30:
            if price_above_ma200_pct > 40:
                risk_score += 2
                signal_reasons.append('Price >40% above 200-MA')
            else:
                risk_score += 1
                signal_reasons.append('Price >30% above 200-MA')
            if signal != 'Sell':
                signal = 'Sell'

        # 🟡 VALUATION SELL SIGNALS

        pe_ratio = fundamentals['pe_ratio']
        pb_ratio = fundamentals['pb_ratio']
        div_yield = fundamentals['dividend_yield']
        hist_div_yield = fundamentals['hist_dividend_yield']
        earnings_growth = fundamentals['earnings_growth']

        # P/E ratio > 2x industry average
        industry_pe = INDUSTRY_PE_AVERAGES.get(ticker, 20)  # Default to 20 if not found
        if pe_ratio and pe_ratio > (2 * industry_pe):
            risk_score += 2
            signal_reasons.append(f'P/E >{2*industry_pe} (High vs Industry)')
            signal = 'Sell'
        elif pe_ratio and pe_ratio > (1.5 * industry_pe):
            risk_score += 1
            signal_reasons.append(f'P/E >{1.5*industry_pe} (Above Industry)')

        # P/B ratio > 2.5-3 without strong earnings growth
        if pb_ratio and pb_ratio > 2.5:
            if earnings_growth is None or earnings_growth < 15:  # Weak earnings growth
                if pb_ratio > 3:
                    risk_score += 2
                    signal_reasons.append('High P/B >3 + Weak Earnings')
                else:
                    risk_score += 1
                    signal_reasons.append('High P/B >2.5 + Weak Earnings')
                signal = 'Sell'

        # Dividend yield drop (indicates price surge)
        if div_yield and hist_div_yield and hist_div_yield > div_yield:
            yield_drop_pct = ((hist_div_yield - div_yield) / hist_div_yield) * 100
            if yield_drop_pct > 30:  # Significant drop
                risk_score += 1
                signal_reasons.append(f'Dividend Yield Drop {yield_drop_pct:.0f}%')

        # Strong sell if multiple risk factors
        if risk_score >= 4:
            signal = 'Strong Sell'
        elif risk_score >= 2:
            signal = 'Sell'

        # Format output values safely
        def safe_format(value, decimal_places=1):
            if value is None:
                return 'N/A'
            try:
                return round(float(value), decimal_places)
            except:
                return 'N/A'

        def safe_format_percent(value, decimal_places=1):
            if value is None:
                return 'N/A'
            try:
                return f"{float(value):.{decimal_places}f}%"
            except:
                return 'N/A'

        return {
            'Ticker': ticker,
            'Date': date_str,
            'Close': safe_format(close, 2),
            'MA_200': safe_format(ma_200, 2),
            'Price_vs_MA200': safe_format_percent(price_above_ma200_pct, 1),
            'RSI': safe_format(rsi, 1),
            'MACD': safe_format(macd, 3),
            'MACD_Signal': safe_format(macd_signal, 3),
            'MACD_Divergence': '🔴 YES' if macd_divergence else '🟢 NO',
            'P/E': safe_format(pe_ratio, 1),
            'P/E_vs_Industry': safe_format(pe_ratio / industry_pe, 1) if pe_ratio else 'N/A',
            'P/B': safe_format(pb_ratio, 1),
            'Div_Yield': safe_format_percent(div_yield, 1),
            'Hist_Div_Yield': safe_format_percent(hist_div_yield, 1),
            'Earnings_Growth': safe_format_percent(earnings_growth, 1),
            'Volume_Ratio': safe_format(volume_ratio, 1),
            'High_Volume_Alert': '🔴 YES' if high_volume_alert else '🟢 NO',
            'Risk_Score': f"{risk_score}/8",
            'Signal': signal,
            'Reasons': ', '.join(signal_reasons) if signal_reasons else 'No sell signals'
        }

    except Exception as e:
        print(f"Error analyzing {ticker}: {e}")
        import traceback
        traceback.print_exc()
        return None

def main():
    """Main function to run the sell signal analysis"""
    print("Starting SELL SIGNAL analysis...")
    print(f"Analyzing stocks: {', '.join(WATCHLIST)}")
    print("-" * 50)

    results = []
    for ticker in WATCHLIST:
        print(f"Processing {ticker}...")
        try:
            df = fetch_technical_data(ticker)
            if df is not None and not df.empty:
                result = check_sell_signals(ticker, df)
                if result:
                    results.append(result)
                    print(f"✓ {ticker} processed successfully")
                else:
                    print(f"✗ {ticker} analysis failed")
            else:
                print(f"✗ {ticker} data fetch failed")
        except Exception as e:
            print(f"✗ Error processing {ticker}: {str(e)}")
            continue

    if not results:
        print("No results to display. Please check your internet connection and try again.")
        return

    # Create results DataFrame
    df_results = pd.DataFrame(results)

    print("\n" + "="*140)
    print("SELL SIGNAL ANALYSIS - TECHNICAL & VALUATION TRIGGERS")
    print("="*140)

    # Display results
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 25)

    print(df_results.to_string(index=False))

    # Summary statistics
    sell_signals = df_results[df_results['Signal'].isin(['Sell', 'Strong Sell'])]
    strong_sell_signals = df_results[df_results['Signal'] == 'Strong Sell']
    high_risk_stocks = df_results[df_results['Risk_Score'].str.extract('(\d+)').astype(int)[0] >= 3]
    overbought_stocks = df_results[df_results['RSI'].astype(str).str.extract('(\d+\.?\d*)').astype(float)[0] > 70]

    print(f"\n📊 SUMMARY:")
    print(f"Total stocks analyzed: {len(df_results)}")
    print(f"Sell signals (all): {len(sell_signals)}")
    print(f"Strong Sell signals: {len(strong_sell_signals)}")
    print(f"High risk stocks (Score ≥3): {len(high_risk_stocks)}")
    print(f"Overbought stocks (RSI>70): {len(overbought_stocks)}")

    if len(strong_sell_signals) > 0:
        print(f"\n🔴 STRONG SELL SIGNALS (High Risk):")
        for _, stock in strong_sell_signals.iterrows():
            print(f"  • {stock['Ticker']}: ₹{stock['Close']} | Risk Score: {stock['Risk_Score']} | Reasons: {stock['Reasons']}")

    if len(sell_signals) > 0:
        print(f"\n📉 ALL SELL SIGNALS:")
        for _, stock in sell_signals.iterrows():
            print(f"  • {stock['Ticker']}: ₹{stock['Close']} | RSI: {stock['RSI']} | P/E: {stock['P/E']} | P/B: {stock['P/B']}")

    # Risk warnings
    if len(high_risk_stocks) > 0:
        print(f"\n⚠️  HIGH RISK STOCKS (Consider reducing position):")
        for _, stock in high_risk_stocks.iterrows():
            if float(str(stock['RSI']).replace('N/A', '50')) > 75:
                print(f"  • {stock['Ticker']}: Severely Overbought (RSI: {stock['RSI']})")
            if 'High P/E' in str(stock['Reasons']):
                print(f"  • {stock['Ticker']}: Overvalued (P/E vs Industry: {stock['P/E_vs_Industry']}x)")

    print("\n" + "="*50)
    print("⚠️  DISCLAIMER: This is for educational purposes only.")
    print("Always do your own research before making investment decisions.")
    print("="*50)
    print("\nSell signal analysis complete!")

if __name__ == '__main__':
    main()

Starting SELL SIGNAL analysis...
Analyzing stocks: INFY.NS, TCS.NS, RELIANCE.NS
--------------------------------------------------
Processing INFY.NS...
✓ INFY.NS processed successfully
Processing TCS.NS...
✓ TCS.NS processed successfully
Processing RELIANCE.NS...
✓ RELIANCE.NS processed successfully

SELL SIGNAL ANALYSIS - TECHNICAL & VALUATION TRIGGERS
     Ticker       Date  Close  MA_200 Price_vs_MA200  RSI   MACD  MACD_Signal MACD_Divergence  P/E  P/E_vs_Industry   P/B Div_Yield Hist_Div_Yield Earnings_Growth  Volume_Ratio High_Volume_Alert Risk_Score Signal                     Reasons
    INFY.NS 2025-05-30 1562.7 1755.88         -11.0% 41.4 14.090       11.951            🟢 NO 24.1              1.0 567.0    403.0%            N/A          -12.7%           1.7             🔴 YES        2/8   Sell High P/B >3 + Weak Earnings
     TCS.NS 2025-05-30 3463.4 3948.20         -12.3% 33.1  7.783       12.971            🟢 NO 25.8              1.0  13.2    173.0%            N/A           -1.7

In [1]:
import smtplib
from email.mime.text import MIMEText

def send_email(subject, body, to_email):
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = 'pantaman44@gmail.com'
    msg['To'] = to_email

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login('pantaman44@gmail.com', 'tsdi ulow fzrt xpsi')
        smtp.send_message(msg)

# Your script logic here
summary = "Your stock strategy result"

send_email("Daily Report", summary, "your_email@gmail.com")
